# Airline Customer Feedback Analysis

### Business Understanding

The airline industry is highly competitive and customer service is a critical area that can influence customer loyalty and brand reputation. Timely and effective handling of customer feedback is crucial for maintaining service quality. Analyzing customer feedback can provide insights into common concerns and areas for improvement.

### Project Goals

The primary goal of this project is twofold:

1. **Sentiment Analysis**: To automatically determine the sentiment of customer comments. The sentiment can be:
   - **Negative (Complaint)**: Indicates the customer faced an issue or is dissatisfied.
   - **Positive (Congratulation)**: Reflects customer satisfaction or a positive experience.
   - **Neutral (Opinion)**: Represents a customer's opinion that is neither a direct complaint nor praise.

2. **Complaint Classification**: To classify customer feedback into categories for routing to the appropriate department. This classification will help in streamlining the process of addressing customer feedback efficiently. Example categories include:
   - **Luggage Issues**: For complaints or feedback related to damaged or lost luggage.
   - **Flight Delays**: For issues pertaining to delayed or cancelled flights.
   - **Billing Issues**: For concerns related to overcharges, refunds, or payment discrepancies.

The end product will be a model or a set of models that can accurately tag incoming customer comments with the correct sentiment and category, allowing the customer service team to promptly address and resolve the issues.

In [47]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
import os.path
import json

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import little_mallet_wrapper as lmw


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline


# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

path_to_mallet = 'mallet-2.0.8/bin/mallet'  # CHANGE THIS TO YOUR MALLET PATH

In [48]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/gabot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gabot/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
# Load data
data = pd.read_excel('output.xlsx', engine='openpyxl')

# Change typo in column name
data.rename(columns={'descripciom': 'descripcion'}, inplace=True)


In [50]:
# Convert to list
data = data.descripcion.values.tolist()

# Remove non-string elements
data = [str(sent) for sent in data]

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['Gracias por escribirnos, hemos recibido tu correo y en seguida daremos '
 'respuesta. Para emergencias, por favor comunicate al 4777920450 ']


In [51]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['gracias', 'por', 'escribirnos', 'hemos', 'recibido', 'tu', 'correo', 'en', 'seguida', 'daremos', 'respuesta', 'para', 'emergencias', 'por', 'favor', 'comunicate', 'al']]


In [52]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['gracias', 'por', 'escribirnos_hemos_recibido', 'tu', 'correo', 'en', 'seguida_daremos_respuesta', 'para', 'emergencias', 'por', 'favor', 'comunicate', 'al']


In [59]:
# Define initial stop words sets
stop_words_en = set(stopwords.words('english'))
stop_words_sp = set(stopwords.words('spanish'))

# Add additional Spanish stopwords
new_stopwords = []
with open('list_stopwords.txt', 'r', encoding='utf-8') as file:
    new_stopwords = json.load(file)

# Combine all stopwords
all_stopwords = stop_words_en.union(stop_words_sp).union(set(new_stopwords))

# Define stopwords excluding certain words if needed
not_stopwords = set() # This should be a set of words you want to exclude from the stop words
final_stop_words = set([word for word in all_stopwords if word not in not_stopwords])

print(final_stop_words)

{'him', 'unos', 'be', 'estás', 'seamos', 'todo', 'estada', 'esa', 'hacer', 'shan', 'once', 'hubieras', 'yourself', 'a', 'hayan', 'algo', 'aren', 'os', 'tenías', 'itinerary_assets', 'ti', 'ct_sendgrid', 'ma', 'seréis', 'tiene', 'tuviste', 'have', 'as', 'was', 'haven', 'mío', 'bz', 'again', 'any', 'poco', 'saber', 'e', 'tuyo', 'hoy', 'teníamos', 'at', 'habrán', 'estuvisteis', 'isn', 'uno', 'fuiste', 'text', 'estuviésemos', 'are', 'width', 'hasta', 'mas', 'estuvo', 'esté', 'px', 'yo', "it's", 'noche', 'what', 'favor', '01d57aa3', 'evehu', 'fui', "shouldn't", 'by', 'those', 'to', 'algunos', 'días', 'so', 'more', 'tenidos', 'hemos', 'tuvimos', 'eso', 'he', 'tus', 'estuvieses', 'el', 'them', 'who', 'estaríais', 'qué', 'estad', 'más', 'fuera', 'tengamos', "should've", 'ante', "wouldn't", 'otro', 'estuvimos', 'in', 'vosotros', 'own', 'estén', 'such', 'size', 'sean', 'podrás', 'mi', 'estaba', 'youtube', 'eras', 'habías', 'seáis', 'electronico', "hasn't", 'que', 'm', 'tuviera', 'había', 'fls_fcl

In [60]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    # return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    return [[word for word in simple_preprocess(str(doc)) if word not in final_stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [61]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [62]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]]


In [63]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('comunicate', 1),
  ('emergencias', 1),
  ('escribirnos', 1),
  ('recibido', 1),
  ('respuesta', 1),
  ('seguida_daremos', 1)]]

In [64]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [37]:
# Save LDA model
lda_model.save("lda_model")

# Optionally, you can also save the corpus and id2word dictionary for later use
# Save corpus
gensim.corpora.MmCorpus.serialize('corpus.mm', corpus)

# Save id2word dictionary
id2word.save('id2word.dict')

In [16]:
# Load LDA model
lda_model = gensim.models.ldamodel.LdaModel.load("lda_model")

# Load corpus
corpus = gensim.corpora.MmCorpus('corpus.mm')

# Load id2word dictionary
id2word = gensim.corpora.Dictionary.load('id2word.dict')

In [38]:
# Print the Keyword in the 4 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.011*"protection_outlook" + 0.010*"reserved" + 0.006*"data" + '
  '0.006*"fls_fclick" + 0.006*"entrada" + 0.006*"detalles" + 0.005*"furl" + '
  '0.005*"nam_safelinks" + 0.005*"evehu" + 0.005*"fupn_dhn"'),
 (1,
  '0.025*"vuelo" + 0.016*"mxn" + 0.014*"itinerary_assets" + 0.010*"hn_evehu" + '
  '0.008*"fzkv_jo" + 0.007*"salida" + 0.007*"reservacion" + 0.006*"horas" + '
  '0.005*"uso" + 0.005*"pixel_gif"'),
 (2,
  '0.007*"yiv_yiv" + 0.003*"yiv" + 0.003*"color" + 0.003*"line" + '
  '0.002*"begin" + 0.002*"right" + 0.002*"serif" + 0.002*"sans" + '
  '0.002*"border" + 0.002*"padding"'),
 (3,
  '0.016*"vuelo" + 0.008*"cotizacion" + 0.006*"julio" + 0.006*"reservacion" + '
  '0.006*"viaje" + 0.005*"fecha" + 0.005*"nombre" + 0.004*"mexico" + '
  '0.004*"pago" + 0.004*"clave"')]


In [34]:
len(texts)

102966